In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif

In [3]:
file_path = r'D:\Coding_Dev\College\SEM_6\machine-learning-niit\Assets\archive\Billionaires Statistics Dataset.csv'
df = pd.read_csv(file_path)

In [4]:
df

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,6.705989e+07,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,2540,1000,Healthcare,Yu Rong,51.0,China,Shanghai,Health clinics,Healthcare,China,...,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397
2636,2540,1000,Food & Beverage,"Richard Yuengling, Jr.",80.0,United States,Pottsville,Beer,Food & Beverage,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
2637,2540,1000,Manufacturing,Zhang Gongyun,60.0,China,Gaomi,Tyre manufacturing machinery,Manufacturing,China,...,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397
2638,2540,1000,Real Estate,Zhang Guiping & family,71.0,China,Nanjing,Real estate,Real Estate,China,...,2.9,"$19,910,000,000,000",50.6,100.2,77.0,9.4,59.2,1.397715e+09,35.861660,104.195397


In [5]:
df['class'] = df['country'].apply(lambda x: 1 if x == 'United States' else 0)

X = df.drop('class', axis=1)
y = df['class']

X = X.drop(["source", "industries", "organization", 'title','state',
            'residenceStateRegion',"city", "birthDate",'birthMonth', 'birthDay', 
            'lastName', 'firstName', 'birthYear', 'date', 'latitude_country',
            'longitude_country', 'personName',"country"], axis=1)
X['gdp_country'] = X['gdp_country'].str.replace(',', '').str.replace(r'\$$', '', regex=True).str.replace('$', '').astype(float)

C:\Users\barar\AppData\Local\Temp\ipykernel_5264\2883355420.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X['gdp_country'] = X['gdp_country'].str.replace(',', '').str.replace(r'\$$', '', regex=True).str.replace('$', '').astype(float)


In [6]:
label_encoders = {}
features = ['category', 'gender', 'countryOfCitizenship', 'selfMade', 'status']

for feature in features:
    label_encoders[feature] = LabelEncoder()
    X[feature] = label_encoders[feature].fit_transform(X[feature])

SI = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(SI.fit_transform(X), columns=X.columns)

In [7]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)

In [8]:
mi_scores = mutual_info_classif(X_scaled, y)

mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mi_scores})

sorted_mi_scores = mi_df.sort_values(by='Mutual Information', ascending=False)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [13]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9715909090909091


In [11]:
coefficients = model.coef_.flatten()
odds_ratios = np.exp(coefficients)
feature_names = X_train.columns
odds_ratio_df = pd.DataFrame({'Feature': feature_names, 'Odds Ratio': odds_ratios})
sorted_odds_ratios = odds_ratio_df.sort_values(by='Odds Ratio', ascending=False)
print(sorted_odds_ratios)

                                       Feature  Odds Ratio
10                                 gdp_country   12.392892
4                         countryOfCitizenship    9.259537
11         gross_tertiary_education_enrollment    3.319064
9                           cpi_change_country    2.964857
1                                   finalWorth    1.654110
5                                     selfMade    1.344213
6                                       status    1.097833
7                                       gender    1.020765
12  gross_primary_education_enrollment_country    1.014517
2                                     category    0.823067
16                          population_country    0.809749
13                     life_expectancy_country    0.785931
0                                         rank    0.739849
3                                          age    0.580087
8                                  cpi_country    0.552259
14                 tax_revenue_country_country    0.5223

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1) 
sns.barplot(x='Odds Ratio', y='Feature', data=sorted_odds_ratios, palette='Blues_d')
plt.title('Sorted Odds Ratios')

plt.subplot(1, 2, 2) 
sns.barplot(x='Mutual Information', y='Feature', data=sorted_mi_scores, palette='Greens_d')
plt.title('Sorted MI Scores')

plt.tight_layout()
plt.show()
